If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [4]:
! pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 17.7 MB/s eta 0:00:0000:0100:01
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.7/158.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 16.4 MB/s eta 0:00:00


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [1]:
import transformers

print(transformers.__version__)

4.19.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/question-answering).

# Fine-tuning a model on a question-answering task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context. We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

![Widget inference representing the QA task](https://github.com/huggingface/notebooks/blob/master/examples/images/question_answering.png?raw=1)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [2]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
#model_checkpoint = "monologg/koelectra-small-v3-finetuned-korquad"
#model_checkpoint = "monologg/koelectra-base-v3-finetuned-korquad"
model_checkpoint = "monologg/kobigbird-bert-base"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [3]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [SQUAD dataset](https://rajpurkar.github.io/SQuAD-explorer/). The notebook should work with any question answering dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [4]:
import json
def load_json(path):
    with open(path, 'r', encoding='utf8') as f:
        return json.load(f)
data = load_json('dataset/train.json')

In [5]:
def read_squad(mode, data, debug=False):
    contexts = []
    questions = []
    question_ids = []
    answers = []
    data = data.copy()

    # train - val split
    if mode == 'train':
        data['data'] = data['data'][:-1*int(len(data['data'])*0.1)]
    elif mode == 'val':
        data['data'] = data['data'][-1*int(len(data['data'])*0.1):]


    till = 100 if debug else len(data['data'])


    for group in data['data'][:till]:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if mode == 'test':
                    contexts.append(context)
                    questions.append(question)
                    question_ids.append(qa['question_id'])
                else: # train or val
                    for ans in qa['answers']:
                        contexts.append(context)
                        questions.append(question)

                        if qa['is_impossible']:
                            answers.append({'text':[''],'answer_start':[-1]})
                        else:
                            #answers.append([ans])
                            answers.append({'text':[ans['text']],'answer_start':[ans['answer_start']]})
    # return formatted data lists
    return contexts, questions, answers, question_ids

In [6]:
from datasets import Dataset
#import pandas as pd
contexts, questions, answers, question_ids = read_squad('train', data)
ids = []
for i in range(len(contexts)):
    ids.append(i)
train_dataset = Dataset.from_dict({'id':ids, 'context': contexts, 'question':questions, 'answers':answers})
    
#df = pd.DataFrame({'context': contexts, 'question': questions, 'answers': answers})
#df.to_csv('train.csv', index=False)
contexts, questions, answers, question_ids = read_squad('val', data)
ids = []
for i in range(len(contexts)):
    ids.append(i)
val_dataset = Dataset.from_dict({'id':ids, 'context': contexts, 'question':questions, 'answers':answers})
#df = pd.DataFrame({'context': contexts, 'question': questions, 'answers': answers})
#df.to_csv('val.csv', index=False)

In [7]:
question_ids

[]

In [8]:
#datasets = load_dataset("squad_v2" if squad_v2 else "squad")
#dataset = load_dataset('json', data_files='dataset/train.json', field='data')
#datasets = load_dataset('csv', data_files={'train':'train.csv', 'validation': 'val.csv'})
#datasets = load_dataset('csv', data_files={'train':'train.csv', 'validation': 'val.csv'})

In [9]:
from datasets import DatasetDict
datasets = DatasetDict({'train':train_dataset, 'validation':val_dataset})

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [10]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 25262
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 2762
    })
})

We can see the training, validation and test sets all have a column for the context, the question and the answers to those questions.

To access an actual element, you need to select a split first, then give an index:

In [11]:
datasets["train"][0]

{'id': 0,
 'context': '이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재의 경쟁력보다는 미래의 경쟁력을 평가하도록 평가의 대상을 확대해야 한다. 지금까지의 경쟁력 평가가 현위치를 중심으로 한 것이었다면, 앞으로의 대안은 과정과 경로를 중시하는 것이 되어야 한다는 것이다. 사실 지식정보화사회에서 가진 것이란 허망한 것이다. 현재의 기술수준이 높더라고, 이를 끊임없이 개량하고 새로운 기술을 발전시킬 수 있는 능력이 없다면 미래의 전망은 어둡다. 기술이나 정보는 실물 자산보다 진부화율이 매우 높다. 둘째, 기술경쟁력 평가를 순위를 매기는 작업이 아니라 이를 통하여 장점과 단점을 파악하는 과정으로 활용하여야 한다. 순위는 대중적인 흥미는 끌 수 있지만, 그것으로부터 교훈이 도출되는 것은 아니다. 기술경쟁력의 평가를 통해서 외국시스템의 장점을 배우고, 한국 시스템의 단점을 교정하는 대안이 발견될 수 있다. 경쟁이란 남과 비교하는 것을 의미한다. 내가 아무리 잘 하더라도 남이 나보다 더 잘 한다면 경쟁에서 진다. 그래서 경쟁은 각박하지만, 과학기술은 경쟁을 피할 수 있는 분야가 아니다. 경쟁 상대국을 연구하고, 경쟁 상대국을 앞설 수 있는 방안을 강구하기 위해서 기술경쟁력에 관한 연구가 더욱 활발해져야 할 것이다.',
 'question': '경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활발해져야 할 연구는 뭐지',
 'answers': {'answer_start': [603], 'text': ['기술경쟁력']}}

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [12]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(datasets["train"])

,id,context,question,answers
0,9123,"현재 3D 프린팅 기술의 급격한 발전으로 3D 프린팅 기술은 기존 산업과의 융·복합 되어 성장 중이며, 기존 단순 시제품(Prototype)제작에서 수요자의 니즈에 맞는 맞춤형 디자인 형상 및 복합 소재 구현 등 기존의 기술이 융·복합되어 새로운 제조 기술로 발전 중이다. 특히, 3D 프린팅 기술은 다품종 소량생산에 적합하고 환자 맞춤형 제작이 가능하기 때문에 의료분야에서의 활용성이 매우 크다. 3D 프린터의 FDM(Fused Deposition Modeling) 기술로 열가소성 생체적합 고분자를 이용한 의료용 지지체의 직접 제작이 가능하게 되면서 생체·의공학 분야에서 본격적으로 활용되기 시작했다. 3D 프린터를 활용하여 보청기, 틀니, 의족, 의수 등 개인 맞춤형 의료 보형물 제작에 활용되고 있고, 환자의 해부용 가상 신체를 제작하여 수술 시뮬레이션용으로 활용함으로써 수술로 인한 의료사고를 사전에 예방하거나, 수술시간의 단축과 수술의 성공률 향상에 기여할 수 있게 되었으며, 의료분야 종사자의 교육에도 활용하고 있다.",환자의 해부용 가상 신체를 제작하여 수술 시뮬레이션용으로 활용함으로 의료사고를 사전에 예방하고 수술시간의 단축에 영향을 준 기술은 뭐니,"{'answer_start': [3], 'text': ['3D 프린팅 기술']}"
1,6941,"강원도 지역의 안보관광 자원들은 군 통제지역 깊숙하게 위치하여 군의 통제와 관리가 인천·경기권의 군부대보다 통제와 관리체계가 더욱 삼엄하다. 또한, 경기도와 달리 강원도는 배치되어 있는 군부대의 수가 많은 데 사단마다 요청사항이 상이하여 군과의 협조에 어려움을 안고 있다. 인천권역은 DMZ 접경지역 내에서 이루어지는 각종 시설이나 습지개발, 교량 시설 등의 개발 계획들은 관할 군부대인 해병 2사단으로부터 군사작전 영향평가를 받는데, 평가 후 합의를 보는데 10년까지 걸리기도 한다. 특히 강화도의 경우 각종 개발계획 진행 시 국방부뿐만 아니라 문화 재청에서도 중첩규제를 받아 개발이 어려운 것으로 나타났다. 지자체는 군부대의 입장통제와 개발 제한 문제는 인천권역의 사회기반시설 구축에 따른 접근성 완화와 지역관광 활성화로 자연히 군부대의 접근 통제 문제도 완화될 것으로 기대하고 있는 것으로 나타났다. 이에 따라 교통 인프라의 구축과 관리적 방안에 대한 정부차원의 지원이 필요한 것으로 나타났다.",개발계획을 진행할 때 국방부의 규제와 문화재청의 통제를 함께 받아 개발이 힘든 지역이 어딜까,"{'answer_start': [275], 'text': ['강화도']}"
2,18467,"바호주(Jose Manuel Barroso) Ⅱ기 집행위원회와 롬푀이(Herman Van Rompuy) 유럽연합(EU: European Union) 정상회의 상임의장, 그리고 애쉬튼(Catherine Ashton) 외교안보 고위대표가 이끈 2009~2014년 EU 지도부는 유럽통합의 제도적·정치적·경제적·안보적 위기 노출에 직면하며 그 임기를 마감하였다. 이 시기에 EU는 경제적으로는 유로존 위기와 각국의 재정위기를 겪으면서 EU 전체의 경쟁력 제고와 유로존의 제도적 오류를 개혁하고 재정 통합에서 진전을 보였다. 제도적으로는 대부분의 EU 회원국이 참여하여 신 재정협약을 수립하였다. EU 대외관계청이 본격적으로 가동되고, 2009년 12월 발효된 리스본 조약에서 이행 기한을 두었던 EU의 의사결정상의 여러 변화들이 실현되었다. 정치적으로 이 시기는 유럽통합에 대한 의문이 대내적으로 제기된 해였다. 각국 선거에서 유럽 회의주의가 팽배하였고, 이러한 경향은 2014년 5월 치러진 유럽의회 선거에서 반 EU 정당이 영국과 프랑스를 중심으로 부상함에 따라, 유럽 회의주의가 득세를 보이는 것으로 이어졌다.",어느 나라들을 중심으로 2014년 5월의 유럽의회 선거에서 반 EU 정당이 떠올랐을까,"{'answer_start': [516], 'text': ['영국과 프랑스']}"
3,12135,"다른 한편으로, 디지털 전환으로 인해 디지털 상거래의 규모가 큰 폭으로 확대되었으며, 외국 사업자의 국내 시장 진출은 빠르게 이행될 것으로 예상됨. 디지털 상거래의 소득 발생 주요 원천은 전통적인 의미의 ‘사업장’이 아니라 ‘사용자 데이터’이므로 물리적 실체를 기반으로 한 현행 과세기준 아래에서는 소득의 원천국가가 과세권을 갖지 못하는, 이른바 ‘국가 간 과세권 배분의 왜곡’ 현상이 발생함. 그뿐만 아니라, 다국적 기업이 인위적으로 세율이 낮은 국가로 본사를 이전하여 조세를 회피하게 되는 경우, 세원 잠식 및 소득 이전(Base erosion and profit shifting)의 문제가 발생하게 됨. 우리나라는 2014년 부가가치세법 제53조의 2 【전자적 용역을 공급하는 국외 사업자의 용역 공급과 사업자등록 등에 관한 특례】를 제정하여 국외 사업자가 제공하는 디지털 서비스에 대한 조세제도를 일부 정비한 바 있으나, 소득세 등에 대한 보다 포괄적인 법제적 정비는 아직 이루어지고 있지 못함.",국가 간 과세권 배분의 왜곡 현상은 무엇으로 인해 확대되었지,"{'answer_start': [9], 'text': ['디지털 전환']}"
4,22142,"결국, 원 기록매체를 압수할 것인가, 본 조의 처분을 할 것인지 여부는 압수를 행하는 자의 재량에 맡겨져 있다고 해석된다. 또한, 판사가 영장을 발부함에 있어 집행방법에 관하여 조건(전자적 기록이 저장된 기록매체를 압수하는 경우 본 조의 처분에 따라야 한다는 조건 등)을 부가하는 것이 가능할 수 있을 것이나, 압수의 집행 방법은 압수 현장의 상황에 따라 좌우되는 면이 큰 이상, 압수의 구체적인 방법은 압수를 행하는 자의 판단에 맡기는 것이 적당한 경우가 많을 것이다. 결국 압수 현장에서의 본 조의 처분에 따른 압수를 할 것인지는 물론, 본 조의 처분에 따라 압수를 하는 경우에도, 수사기관이 직접 압수를 할 것인지, 피압수자에게 이를 하게 할 것인지 여부, ‘복사’, ‘인쇄’, ‘이전’ 중 어떤 처분을 할 것인지에 대하여도 압수 현장의 상황에 따라 압수를 행하는 자의 판단에 따라 이를 결정하게 될 것이다.",영장은 누구를 통해 발급되지,"{'answer_start': [73], 'text': ['판사']}"
5,9767,"비용이나 세금은 심리학적으로 부담을 주는 용어이다. 내 주머니에서 돈이 나가야 하고 내 봉급이 깎인다는 생각을 하게 되어 새로운 세대들의 심리적 구조 및 사유체계와는 잘 맞지 않는 개념이다. 즉, 국가가 지향하는 가치 및 목표와 국민들의 가치관 사이에 괴리감이 생기기 쉽다. 국민들에게 심리적인 측면을 설명할 수 있어야 세금이나 비용이라는 용어가 국민들에게 받아들여지지 않을까 생각한다. 과거의 경우에도 통일비용을 이야기하면 할수록 국민들이 통일을 싫어하는 구조가 잉태되었다. 지금 통일세를 이야기하면 국민들 사이에 통일을 꺼리는 상황이 발생할 수도 있다. 독일에서는 통일 이전에 법학 전공자들의 실업률이 매우 높았는데, 통일 이후에 가장 선호하는 전공으로 바뀌었다. 통일 이후에 법률 전문가에 대한 수요가 급증하는 바람에 법학 전공자들은 모두 일자리를 얻게 되었던 것이다. 이러한 측면에서 북한주민의 심리를 고려하는 것이 필요하다.",통일 이전에 법학 전공자들의 실업률이 매우 높았던 나라가 어디야,"{'answer_start': [312], 'text': ['독일']}"
6,18386,"발전원가의 발표에 있어서 추가적으로 고려할 사항은 신재생에너지 발전원의 발전단가가 지속 하락하고 있다는 점이다. 산업통상자원부에서 2014년 9월에 발표 한 ｢제4차 신재생에너지보급계획｣에 따르면 신재생에너지의 발전단가는 2013년에 육상풍력이 이미 가스발전보다 저렴해졌다. 2017년에는 해상풍력과 태양광이 가스발전과 유사한 수준이 되며 2022년에는 태양광과 해상풍력도 가스발전보다 낮은 발전단가를 갖게 될 것으로 예상하고 있다. 신재생에너지의 발전단가가 기존 발전원과 경쟁력을 갖추는 그리드패리티(grid parity)가 7차 전력계획의 계획기간 내에 발생하는 것이다. 반면 기존 발전원의 발전단가는 지속 상승하고 있다. 1차 전력계획이

## Preprocessing the training data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [14]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [15]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on two sentences (one for the answer, one for the context):

In [13]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [2, 30376, 14545, 29023, 11281, 535, 3, 19611, 11281, 14545, 555, 24615, 4551, 12371, 518, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [16]:
#max_length = 384 # The maximum length of a feature (question and context)
#doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
#max_length = 768 # The maximum length of a feature (question and context)
#doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [25]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 512:
        print('Found!')
        break
example = datasets["train"][i]

Without any truncation, we get the following length for the input IDs:

In [26]:
len(tokenizer(example["question"], example["context"])["input_ids"])

312

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [27]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

312

Note that we never want to truncate the question, only the context, else the `only_second` truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with `return_overflowing_tokens=True` and by passing the stride:

In [28]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Now we don't have one list of `input_ids`, but several: 

In [29]:
[len(x) for x in tokenized_example["input_ids"]]

[312]

And if we decode them, we can see the overlap:

In [24]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] 정부미술품과 정부미화물품 등을 제한 이것은 내용연수 및 감가상각 방법에 따라 감가상각하는데 이것은 뭐야 [SEP] 일반유형자산의 취득원가는 당해 자산의 건설원가 또는 매입가액에 취득부대 비용을 가산한 금액을 취득원가로 하고 있습니다. 다만 무주부동산의 취득, 국가외 상대방과의 교환 및 기부채납 등의 방법으로 자산을 취득할 경우 취득 당시의 공정가액을 취득원가로 하고 있으며, 국가회계실체 또는 중앙관서 간 발생하는 관리전환은 무상거래일 경우 자산의 장부가액을 취득원가로 하고, 유상거래일 경우에는 자산의 공정가액을 취득원가로 하고 있습니다. 일반유형자산에 사용수익권이 설정된 경우 이는 해당자산의 차감항목으로 표시하며 사용수익권은 수증자가 무형자산으로 인식하는 금액과 동일하게 평가하고 있습니다. 일반유형자산의 내용연수를 연장시키거나 가치를 실질적으로 증가시키는 지출은 자산의 증가로 처리하고, 원상회복 시키거나 능률유지를 위한 지출은 비용으로 처리하고 있습니다. 토지, 입목, 정부미술품 · 정부미화물품을 제외한 일반유형자산은 아래의 내용연수 및 감가상각 방법에 따라 감가상각하며, 감가상각누계액에 가산하여 자산의 차감 형식으로 표시하고 있습니다. [SEP]


Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an `offset_mapping`:

In [25]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 2), (2, 4), (4, 5), (5, 6), (7, 9), (9, 10), (10, 12), (12, 13), (14, 15), (15, 16), (17, 19), (20, 22), (22, 23), (24, 26), (26, 28), (29, 30), (31, 35), (36, 38), (38, 39), (40, 42), (43, 47), (47, 48), (48, 50), (51, 53), (53, 54), (55, 56), (56, 57), (0, 0), (0, 2), (2, 3), (3, 4), (4, 6), (6, 7), (8, 10), (10, 11), (11, 12), (12, 13), (14, 16), (17, 19), (19, 20), (21, 23), (23, 24), (24, 25), (26, 28), (29, 31), (31, 32), (32, 33), (33, 34), (35, 37), (37, 39), (40, 42), (42, 43), (44, 46), (46, 47), (48, 50), (50, 51), (52, 54), (54, 55), (55, 56), (56, 57), (58, 60), (61, 62), (62, 63), (63, 65), (65, 66), (67, 69), (70, 72), (72, 73), (73, 75), (75, 76), (77, 79), (79, 80), (81, 83), (83, 84), (85, 88), (88, 89), (89, 90), (91, 93), (94, 95), (96, 98), (98, 99), (99, 100), (101, 102), (102, 103), (104, 106), (106, 108), (109, 111), (111, 112), (113, 115), (115, 116), (117, 119), (120, 122), (123, 125), (125, 126), (127, 129), (129, 130), (130, 131), (131, 132), (1

This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token (`[CLS]`) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [26]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

정부 정부


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the `sequence_ids` method of our `tokenized_example` can be useful:

In [27]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


It returns `None` for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [28]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

275 278


And we can double check that it is indeed the theoretical answer:

In [29]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

일반유형자산
일반유형자산


For this notebook to work with any kind of models, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context):

In [17]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [18]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [19]:
features = prepare_train_features(datasets['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [19]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/26 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [20]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint, attention_type='original_full')

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'bert.pooler.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of 

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [21]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
#    push_to_hub=True,
    push_to_hub=False,
    report_to=[],
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-squad"` or `"huggingface/bert-finetuned-squad"`).

Then we will need a data collator that will batch our processed examples together, here the default one will work:

In [22]:
from transformers import default_data_collator

data_collator = default_data_collator

We will evaluate our model and compute metrics in the next section (this is a very long operation, so we will only compute the evaluation loss during training).

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [23]:
from datasets import load_metric
import numpy as np
metric = load_metric('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #print('###')
    #print(predictions)
    #print('###')
    #print(labels)
    #predictions = np.argmax(predictions, axis=0)
    pred_starts = np.argmax(predictions[0], axis=-1)
    pred_ends = np.argmax(predictions[1], axis=-1)
    pred_comb = pred_starts*10000+pred_ends
    label_comb = labels[0]*10000+labels[1]
    #return metric.compute(predictions=predictions, references=labels)
    #return metric.compute(predictions=pred_starts, references=labels[0])
    return metric.compute(predictions=pred_comb, references=label_comb)

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

We can now finetune our model by just calling the `train` method:

In [25]:
trainer.train()

/home/kotech/venv-trans/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25262
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 15800
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,1.113300,0.495758,0.826575
2,0.450600,0.507528,0.818610
3,0.326200,0.558517,0.813903
4,0.206200,0.646969,0.811007
5,0.155100,0.779217,0.800869
6,0.092700,0.936322,0.803765
7,0.065700,1.139366,0.804127
8,0.042600,1.207541,0.808834
9,0.035700,1.388054,0.810282
10,0.026900,1.511049,0.809196


Saving model checkpoint to kobigbird-bert-base-finetuned-squad/checkpoint-500
Configuration saved in kobigbird-bert-base-finetuned-squad/checkpoint-500/config.json
Model weights saved in kobigbird-bert-base-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in kobigbird-bert-base-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in kobigbird-bert-base-finetuned-squad/checkpoint-500/special_tokens_map.json
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2762
  Batch size = 32
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; wil

/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2762
  Batch size = 32
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to kobigbird-bert-base-finetuned-squad/checkpoint-5000
Configuration saved in kobigbird-bert-base-finetuned-squad/checkpoint-5000/config.json
Model weights saved in kobigbird-bert-base-finetuned-squad/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in kobigbird-bert-base-finetuned-squad/checkpoint-5000/tokenizer_confi

Saving model checkpoint to kobigbird-bert-base-finetuned-squad/checkpoint-9000
Configuration saved in kobigbird-bert-base-finetuned-squad/checkpoint-9000/config.json
Model weights saved in kobigbird-bert-base-finetuned-squad/checkpoint-9000/pytorch_model.bin
tokenizer config file saved in kobigbird-bert-base-finetuned-squad/checkpoint-9000/tokenizer_config.json
Special tokens file saved in kobigbird-bert-base-finetuned-squad/checkpoint-9000/special_tokens_map.json
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2762
  Batch size = 32
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars

Special tokens file saved in kobigbird-bert-base-finetuned-squad/checkpoint-13000/special_tokens_map.json
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2762
  Batch size = 32
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to kobigbird-bert-base-finetuned-squad/checkpoint-13500
Configuration saved in kobigbird-bert-base-finetuned-squad/checkpoint-13500/config.json
Model weights saved in kobigbird-bert-base-finetuned-squad/checkpoint-13500/pytorch

TrainOutput(global_step=15800, training_loss=0.1196317099091373, metrics={'train_runtime': 9371.1294, 'train_samples_per_second': 53.915, 'train_steps_per_second': 1.686, 'total_flos': 1.3934963125690368e+17, 'train_loss': 0.1196317099091373, 'epoch': 20.0})

In [251]:
trainer.train()

/home/kotech/venv-trans/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25262
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 15800


Epoch,Training Loss,Validation Loss
1,0.190700,0.668153
2,0.138200,0.764964
3,0.108100,0.822051
4,0.112700,0.898231
5,0.090400,1.017879
6,0.059900,1.140363
7,0.044800,1.315109
8,0.036600,1.420347
9,0.025900,1.471424
10,0.028800,1.571545


Saving model checkpoint to koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-500
Configuration saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-500/config.json
Model weights saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-500/special_tokens_map.json
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2762
  Batch size = 32
Saving model checkpoint to koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-1000
Conf

tokenizer config file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-5000/tokenizer_config.json
Special tokens file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-5000/special_tokens_map.json
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-5500
Configuration saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-5500/config.json
Model weights saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-5500/pytorch_model.bin
tokenizer config file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-5500/tokenizer_config.json
Special tokens file saved in koelectra-base-

Saving model checkpoint to koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-10000
Configuration saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-10000/config.json
Model weights saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-10000/tokenizer_config.json
Special tokens file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-10000/special_tokens_map.json
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 2762
  Batch size = 32
Saving model checkpoint to koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint

tokenizer config file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-14500/tokenizer_config.json
Special tokens file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-14500/special_tokens_map.json
/home/kotech/venv-trans/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-15000
Configuration saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-15000/config.json
Model weights saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-15000/pytorch_model.bin
tokenizer config file saved in koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-15000/tokenizer_config.json
Special tokens file saved in koelectra

TrainOutput(global_step=15800, training_loss=0.04760223518444013, metrics={'train_runtime': 8005.2872, 'train_samples_per_second': 63.113, 'train_steps_per_second': 1.974, 'total_flos': 1.3201757337329664e+17, 'train_loss': 0.04760223518444013, 'epoch': 20.0})

Since this training is particularly long, let's save the model just in case we need to restart.

In [26]:
trainer.save_model("test-kobigbird-base-trained")

Saving model checkpoint to test-kobigbird-base-trained
Configuration saved in test-kobigbird-base-trained/config.json
Model weights saved in test-kobigbird-base-trained/pytorch_model.bin
tokenizer config file saved in test-kobigbird-base-trained/tokenizer_config.json
Special tokens file saved in test-kobigbird-base-trained/special_tokens_map.json


In [331]:
trainer.save_model("test-kosquad-base-trained")

Saving model checkpoint to test-kosquad-base-trained
Configuration saved in test-kosquad-base-trained/config.json
Model weights saved in test-kosquad-base-trained/pytorch_model.bin
tokenizer config file saved in test-kosquad-base-trained/tokenizer_config.json
Special tokens file saved in test-kosquad-base-trained/special_tokens_map.json


## Evaluation

Evaluating our model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and en position of our answers: if we take a batch from our validation datalaoder, here is the output our model gives us:

In [252]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

The output of the model is a dict-like object that contains the loss (since we provided labels), the start and end logits. We won't need the loss for our predictions, let's have a look a the logits:

In [253]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([32, 512]), torch.Size([32, 512]))

We have one logit for each feature and each token. The most obvious thing to predict an answer for each featyre is to take the index for the maximum of the start logits as a start position and the index of the maximum of the end logits as an end position.

In [254]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([170, 134,  85,  73, 262,  97, 156,  65, 218, 185,  33, 190,  75,  32,
         218, 196, 142,  92,  99, 295, 272,  96,  58, 197,  94,  40,  36, 139,
          19, 134, 169,  47], device='cuda:0'),
 tensor([172, 137,  87,  73, 262,  99, 160,  65, 222, 187,  35, 193,  77,  37,
         219, 203, 144,  94, 103, 296, 275,  97,  59, 201,  97,  40,  38, 140,
          81, 135, 170,  50], device='cuda:0'))

This will work great in a lot of cases, but what if this prediction gives us something impossible: the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

However, picking the second best answer is not as easy as picking the best one: is it the second best index in the start logits with the best index in the end logits? Or the best index in the start logits with the second best index in the end logits? And if that second best answer is not possible either, it gets even trickier for the third best answer.


To classify our answers, we will use the score obtained by adding the start and end logits. We won't try to order all the possible answers and limit ourselves to with a hyper-parameter we call `n_best_size`. We'll pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we will sort them by their score and keep the best one. Here is how we would do this on the first feature in the batch:

In [255]:
n_best_size = 20

In [256]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

And then we can sort the `valid_answers` according to their `score` and only keep the best one. The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [27]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [28]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

  0%|          | 0/3 [00:00<?, ?ba/s]

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [29]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `BigBirdForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BigBirdForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2762
  Batch size = 32


The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing), so we set them back:

In [30]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [31]:
max_answer_length = 30

In [32]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

NameError: name 'output' is not defined

We can compare to the actual ground-truth answer:

In [325]:
datasets["validation"][0]["answers"]

{'answer_start': [261], 'text': ['관치금융']}

Our model picked the right as the most likely answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [33]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer (when `squad_v2 = True`). The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

In [34]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [35]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 2762 example predictions split into 2762 features.


  0%|          | 0/2762 [00:00<?, ?it/s]

Then we can load the metric from the datasets library.

In [36]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [346]:
trainer.train("koelectra-base-v3-finetuned-korquad-finetuned-squad/checkpoint-10000")

AttributeError: 'Trainer' object has no attribute 'from_trained'

epoch 23: em:85.26, f1:90.06

In [37]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 85.40912382331643, 'f1': 90.2480374239969}

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("sgugger/my-awesome-model")
```